Mirror AirNow data
==================

Downloads Airnow's hourly data files, saving them in `./AirNow`.

Reports to stat.createlab.org as `Airnow Hourly Data - Download`.

Docs for the daily data files are here: https://docs.airnowapi.org/docs/HourlyDataFactSheet.pdf

In [0]:
import datetime, dateutil, email, ftplib, glob, json, os, re, time, requests, socket, subprocess, sys, traceback
from dateutil import rrule, tz, parser

In [0]:
# Boilerplate to load utils.ipynb
# See https://github.com/CMU-CREATE-Lab/python-utils/blob/master/utils.ipynb


def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']

    tmpname = '/tmp/%s-%s-%d.py' % (os.path.basename(filename_or_url),
                                    datetime.datetime.now().strftime('%Y%m%d%H%M%S%f'),
                                    os.getpid())
    src = '\n\n\n'.join(src)
    open(tmpname, 'w').write(src)
    code = compile(src, tmpname, 'exec')
    exec(code, globals())


exec_ipynb('./python-utils/utils.ipynb')
exec_ipynb('./airnow-common.ipynb')

In [0]:
STAT_SERVICE_NAME = 'Airnow Hourly Data - Download'
STAT_HOSTNAME = 'hal21'
STAT_SHORTNAME = 'airnow-hourly-data-download'

Stat.set_service(STAT_SERVICE_NAME)

In [0]:
def mirror_timestamp(timestamp):
    filename = timestamp.strftime('%Y%m%d%H.dat')
    src = AirnowCommon.directory_from_date(timestamp) + '/HourlyData_' + filename
    dest = 'AirNow/' + filename
    (is_new, message, status_code) = AirnowCommon.mirror_airnow_file(src, dest)

    if is_new:
        Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    else:
        if status_code == 304:
            print(message) # simply do a print here to reduce noise sent to stat.createlab.org
            # Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        elif status_code < 400:
            Stat.info(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
        else:
            Stat.warning(message, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)


#mirror_timestamp(dateutil.parser.parse('2016-12-31 00:00'))

In [0]:
def compute_first_date_to_check():
    files = glob.glob('AirNow/[0-9]*.dat')
    if len(files) == 0:
        return datetime.datetime(2013, 8, 1)
    last_file = sorted(files)[-1]
    last_date = datetime.datetime.strptime(last_file, "AirNow/%Y%m%d%H.dat")
    Stat.debug('Most recently mirrored data file is %s (%s)' % (last_file, last_date), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    first_date_to_check = last_date - datetime.timedelta(days = 30)
    Stat.debug('Checking for updates starting with date %s' % (first_date_to_check), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
    return first_date_to_check

#compute_first_date_to_check()

In [0]:
def mirror():
    start = compute_first_date_to_check()
    now = datetime.datetime.utcnow()

    timestamps_to_mirror = list(rrule.rrule(rrule.HOURLY, dtstart=start, until=now))
    Stat.info('Mirroring %d data files, starting with %s... (up-to-date files will not be logged here)' % (len(timestamps_to_mirror), start), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)

    for timestamp in timestamps_to_mirror:
        mirror_timestamp(timestamp)

try:
    starting_timestamp = datetime.datetime.now().timestamp()
    mirror()
    elapsed_seconds = datetime.datetime.now().timestamp() - starting_timestamp
    Stat.up('Mirror completed successfully (elapsed time: %d seconds)' % (elapsed_seconds), host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)
except Exception as e:
    Stat.down('Exception recorded %s' % e, host=STAT_HOSTNAME, shortname=STAT_SHORTNAME)